In [1]:
import mlflow
import os

# 1. Verificamos variables de entorno (inyectadas por Docker)
print(f"--> Tracking URI: {os.environ.get('MLFLOW_TRACKING_URI')}")
print(f"--> S3 Endpoint:  {os.environ.get('MLFLOW_S3_ENDPOINT_URL')}")

# 2. Configuración del experimento
mlflow.set_experiment("Test_Inicial_Infraestructura")

# 3. Simulamos un entrenamiento
print("--> Iniciando Run de prueba...")
with mlflow.start_run():
    # Log de parámetros (ej. hiperparámetros)
    mlflow.log_param("tipo_modelo", "RandomForest")
    mlflow.log_param("n_estimators", 100)
    
    # Log de métricas (ej. accuracy)
    mlflow.log_metric("accuracy", 0.98)
    
    # 4. Prueba de escritura en MinIO (S3)
    # Creamos un archivo falso para simular un modelo guardado
    with open("modelo_dummy.txt", "w") as f:
        f.write("Este archivo demuestra que Jupyter puede escribir en MinIO.")
    
    # Lo subimos como "artifact"
    mlflow.log_artifact("modelo_dummy.txt")
    
    print("✅ ¡ÉXITO! Datos enviados a MLflow y archivo subido a MinIO.")

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_config.py:383: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


--> Tracking URI: http://mlflow:5000
--> S3 Endpoint:  http://minio:9000


2026/01/22 11:32:20 INFO mlflow.tracking.fluent: Experiment with name 'Test_Inicial_Infraestructura' does not exist. Creating a new experiment.


--> Iniciando Run de prueba...
✅ ¡ÉXITO! Datos enviados a MLflow y archivo subido a MinIO.


In [2]:
import paho.mqtt.client as mqtt
import time
import json
import random

# Configuración de conexión (Usamos el nombre DNS de Docker)
BROKER = "ind-mqtt" 
PORT = 1883
TOPIC = "planta/sensores"

# Simulamos datos de un motor
def simular_motor():
    client = mqtt.Client()
    try:
        client.connect(BROKER, PORT, 60)
        print(f"📡 Conectado al Broker MQTT ({BROKER}). Enviando telemetría...")
        
        for i in range(5):
            payload = {
                "id_maquina": "prensa_hidraulica_01",
                "temperatura": round(random.uniform(65.0, 85.0), 2),
                "vibracion": round(random.uniform(0.2, 0.9), 2),
                "estado": "OPERATIVO"
            }
            # Convertir diccionario a texto JSON
            mensaje = json.dumps(payload)
            
            # Publicar en el topic
            client.publish(TOPIC, mensaje)
            print(f"Enviado -> {mensaje}")
            time.sleep(2)
            
        print("Fin de la transmisión.")
        
    except Exception as e:
        print(f"❌ Error de conexión: {e}")

simular_motor()

/tmp/ipykernel_483/2165596978.py:13: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


📡 Conectado al Broker MQTT (ind-mqtt). Enviando telemetría...
Enviado -> {"id_maquina": "prensa_hidraulica_01", "temperatura": 77.42, "vibracion": 0.64, "estado": "OPERATIVO"}
Enviado -> {"id_maquina": "prensa_hidraulica_01", "temperatura": 81.78, "vibracion": 0.83, "estado": "OPERATIVO"}
Enviado -> {"id_maquina": "prensa_hidraulica_01", "temperatura": 76.89, "vibracion": 0.22, "estado": "OPERATIVO"}
Enviado -> {"id_maquina": "prensa_hidraulica_01", "temperatura": 84.74, "vibracion": 0.29, "estado": "OPERATIVO"}
Enviado -> {"id_maquina": "prensa_hidraulica_01", "temperatura": 70.97, "vibracion": 0.62, "estado": "OPERATIVO"}
Fin de la transmisión.
